# Flight FAQ Chatbot

## 1. Adding knowledge base to your chatbot

The Chatbot's ability to converse is defined by the data available to it. Take a look at the Ques.txt file to find questions and ans.txt file to find answers to those questions. This chatbot will essentially run a cosine similarity on a question posed with the question bank and try to find an answer. 

Let us begin by importing relevant libraries.

In [1]:
import nltk # to process text data
import numpy as np # to represent corpus as arrays
import random 
import operator
import string # to process standard python strings

# We will use this later to decide how similar two sentences are
from sklearn.metrics.pairwise import cosine_similarity 

# Remember when you built a function to create a tfidf bag of words in Experience 2? This function does the same thing!
from sklearn.feature_extraction.text import TfidfVectorizer 

In [2]:
import os
filepath = os.getcwd()+r'\\SAP FAQ Answers.txt'
corpus = open(filepath,'r',errors = 'ignore')
raw_data_ans = corpus.read()
print (raw_data_ans)

filepath = os.getcwd()+'\\SAP FAQ Questions.txt'
corpus = open(filepath,'r',errors = 'ignore')
raw_data = corpus.read()
print (raw_data)

You can use YM49, be aware that you can only send the same FPV filename once in a day, the "day" for the batch plan tool starts at 4 PM.

What is the transaction to send the FPV?



#  Conversion to lower case

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [3]:
raw_data=raw_data.lower() # converts to lowercase
print (raw_data)

what is the transaction to send the fpv?



####  Segmentation, Lematization and tokenization

We will convert all text to lower case first. Remember to inspect the result once we are done.

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
sent_tokens = nltk.sent_tokenize(raw_data) # converts documents to list of sentences 

print(sent_tokens)

[nltk_data] Error loading punkt: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>
[nltk_data] Error loading wordnet: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\carolina.filippini/nltk_data'
    - 'C:\\Users\\carolina.filippini\\Anaconda3\\nltk_data'
    - 'C:\\Users\\carolina.filippini\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\carolina.filippini\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\carolina.filippini\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [27]:
filepath = os.getcwd()+r'\\Chatbot answers.txt'
corpus = open(filepath,'r',errors = 'ignore')
raw_data_ans = corpus.read()

sent_tokens_ans = nltk.sent_tokenize(raw_data_ans)# converts documents to list of sentences 

print(sent_tokens_ans)

['We accept Visa, MasterCard and American Express credit cards, if you need to pay with two different cards, please call our Contact Center.', 'Yes, the data you provide is encrypted, in this way we can offer you a safe experience with every transaction.', 'No, to guarantee your accommodation, it is necessary to complete your payment with a valid credit card.', 'Yes, every reservation made through our official website or our Contact Center will require to cover 100% of the total payment of the reservation.', 'Hotel Xcaret Arte is located in front of the Caribbean Sea, next to Hotel Xcaret Mexico 15 minutes away from Playa del Carmen 45 minutes away from Cancun International Airport 45 minutes away.', 'Yes for your comfort and enjoyment, the hotels are beach front and have signalized accesses.', 'Hotel Xcaret Mexico and Hotel Xcaret Art own 900 suites and La Casa de la Playa owns 63 suites.', 'Yes, the hotels have ramps and wheelchair access at your disposal, as well as, some suites are

In [28]:
cont = 1
for i in sent_tokens_ans:
    print (cont , '-' , i , '\n')
    cont+= 1

1 - We accept Visa, MasterCard and American Express credit cards, if you need to pay with two different cards, please call our Contact Center. 

2 - Yes, the data you provide is encrypted, in this way we can offer you a safe experience with every transaction. 

3 - No, to guarantee your accommodation, it is necessary to complete your payment with a valid credit card. 

4 - Yes, every reservation made through our official website or our Contact Center will require to cover 100% of the total payment of the reservation. 

5 - Hotel Xcaret Arte is located in front of the Caribbean Sea, next to Hotel Xcaret Mexico 15 minutes away from Playa del Carmen 45 minutes away from Cancun International Airport 45 minutes away. 

6 - Yes for your comfort and enjoyment, the hotels are beach front and have signalized accesses. 

7 - Hotel Xcaret Mexico and Hotel Xcaret Art own 900 suites and La Casa de la Playa owns 63 suites. 

8 - Yes, the hotels have ramps and wheelchair access at your disposal, as w

In [29]:
print(len(sent_tokens))
print(len(sent_tokens_ans))

38
38


In [30]:
res = {sent_tokens[i]: sent_tokens_ans[i] for i in range(len(sent_tokens))} 
print(res)

{'which payment methods are accepted at hotel xcaret mexico, hotel xcaret arte and la casa de la playa?': 'We accept Visa, MasterCard and American Express credit cards, if you need to pay with two different cards, please call our Contact Center.', 'is my credit card data safe at the time of booking my reservation?': 'Yes, the data you provide is encrypted, in this way we can offer you a safe experience with every transaction.', 'could i pay for my reservation at the time of check-in at the hotel?': 'No, to guarantee your accommodation, it is necessary to complete your payment with a valid credit card.', 'will my credit card be charged at the time of booking my reservation on the website or on the phone?': 'Yes, every reservation made through our official website or our Contact Center will require to cover 100% of the total payment of the reservation.', 'where is hotel xcaret mexico, hotel xcaret arte and la casa de la playa located?': 'Hotel Xcaret Arte is located in front of the Carib

In [31]:
word_tokens = nltk.word_tokenize(raw_data)# converts documents to list of words
print (word_tokens)

['which', 'payment', 'methods', 'are', 'accepted', 'at', 'hotel', 'xcaret', 'mexico', ',', 'hotel', 'xcaret', 'arte', 'and', 'la', 'casa', 'de', 'la', 'playa', '?', 'is', 'my', 'credit', 'card', 'data', 'safe', 'at', 'the', 'time', 'of', 'booking', 'my', 'reservation', '?', 'could', 'i', 'pay', 'for', 'my', 'reservation', 'at', 'the', 'time', 'of', 'check-in', 'at', 'the', 'hotel', '?', 'will', 'my', 'credit', 'card', 'be', 'charged', 'at', 'the', 'time', 'of', 'booking', 'my', 'reservation', 'on', 'the', 'website', 'or', 'on', 'the', 'phone', '?', 'where', 'is', 'hotel', 'xcaret', 'mexico', ',', 'hotel', 'xcaret', 'arte', 'and', 'la', 'casa', 'de', 'la', 'playa', 'located', '?', 'are', 'hotel', 'xcaret', 'mexico', ',', 'hotel', 'xcaret', 'arte', 'and', 'la', 'casa', 'de', 'la', 'playa', 'beach', 'front', '?', 'how', 'many', 'suites', 'are', 'there', 'in', 'hotel', 'xcaret', 'mexico', ',', 'hotel', 'xcaret', 'arte', 'and', 'la', 'casa', 'de', 'la', 'playa', '?', 'are', 'hotel', 'xcaret

In [32]:
lemmer = nltk.stem.WordNetLemmatizer() #Initiate lemmer class. WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

In [33]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict))) 

## 2. Adding Chatbot functionality - Cosine similarity

In [34]:
GREETING_INPUTS = ["hello", "hi", "greetings", "sup", "what's up","hey", "hey there"]
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

def greeting(sentence):
    for word in sentence.split(): # Looks at each word in your sentence
        if word.lower() in GREETING_INPUTS: # checks if the word matches a GREETING_INPUT
            return random.choice(GREETING_RESPONSES) # replies with a GREETING_RESPONSE

The functionality of the chatbot is done by creating a loop for running the chatbot. Take a loof at the function below. Each line in the function is important as it calls another function to perform an important step. The function 'response' is responsible for how the chatbot behaves. Having a master function for each functionality is recommended as good programming practise. 


In [35]:
def response(user_response):
    
    robo_response='' # initialize a variable to contain string
    sent_tokens.append(user_response) #add user response to sent_tokens
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english') 
    tfidf = TfidfVec.fit_transform(sent_tokens) #get tfidf value
    vals = cosine_similarity(tfidf[-1], tfidf) #get cosine similarity value
    idx=vals.argsort()[0][-2] 
    flat = vals.flatten() 
    flat.sort() #sort in ascending order
    req_tfidf = flat[-2] 
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response, vals
    else:
        robo_response = robo_response+res[sent_tokens[idx]]
        return robo_response, vals

Finally, let us create the chatbot interface and create a persona around it. Let us call it 'Jane' and use cosine similarity to find FAQs similar to the question asked and answer them. 

In [36]:

flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= response(user_response)
                print(resp[0], )
                sent_tokens.remove(user_response)
                resp_l = resp[1].tolist()
                resp_l[0].pop()
                print(' (With similarity of ',max(resp_l[0]),')')

    else:
        flag=False
        print("Jane: Bye! take care..")
       

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
Can I smoke?
Jane: 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


No, you can't, The suites are non-smoking.
 (With similarity of  0.7590403519285281 )
is there a shuttle?
Jane: The shuttle service (shared) does not require any booking, However, if the suite that you booked offers private service, your confirmation to your Guest Service Manager is necessary within at least 24 hours in advance.
 (With similarity of  0.48572179569147267 )
do you host weddings?
Jane: Allow our flawless service and specialized personnel to contribute in making your dream come true, Please write toÂ customerservice@hotelxcaret.comÂ to start the preparation of the best day of your life.
 (With similarity of  0.4313089872349581 )
are you pet friendly?
Jane: Yes, the hotels have ramps and wheelchair access at your disposal, as well as, some suites are setup for them.
 (With similarity of  0.28754489017777174 )
are you pet-friendly?
Jane: Yes, Hotel Xcaret Mexico, Hotel Xcaret Arte and La Casa de la Playa are pet-friendly, so that you can stay with your pet.
 (With similarity

## 3. Chatbot functionality using Doc2vec

We will be covering one more type of model for creating chatbots. As we saw the cosine similarity model works on using that algorithm to find similarities between 2 sentences. But can we now use a neural network to solve this problem? Lets take a look.

Doc2Vec is a neural network based model that essentially creates vectors out of documents. In order to understand Doc2vec you also need to understand word2vec. 


#### What is word2vec?
It’s a Model to create the word embeddings, where it takes input as a large corpus of text and produces a vector space typically of several hundred dimesions. it was introduced in two papers between September and October 2013, by a team of researchers at Google. The underlying assumption of Word2Vec is that two words sharing similar contexts also share a similar meaning and consequently a similar vector representation from the model.

For instance: “Bank”, “money” and “accounts” are often used in similar situations, with similar surrounding words like “dollar”, “loan” or “credit”, and according to Word2Vec they will therefore share a similar vector representation. 

![Image](img1.png)

#### What is Doc2vec?

So the objective of Doc2vec is to create the numerical representation of sentence/paragraphs/documents unlike word2vec that computes a feature vector for every word in the corpus, Doc2Vec computes a feature vector for every document in the corpus. The vectors generated by Doc2vec can be used for tasks like finding similarity between sentences/paragraphs/documents

<strong> We will be using this property of doc2vec to create our own similarity model.</strong>


Let us begin by importing the relevant libraries.

In [37]:
import subprocess
import sys
subprocess.check_call([sys.executable, "-m", "pip", "install", "gensim == 3.8.1"])

0

In [38]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

The first main aim is to tag our data. The doc2vec model requires us to tag our data to use it effectively. Here is a good learning link for a [starter code](https://www.kaggle.com/fmitchell259/creating-a-doc2vec-model) for doc2vec. The following [link](https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e) can also be usedful and it recommended to read them. 

In [39]:
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(sent_tokens)]

In [40]:
max_epochs = 100
vec_size = 20 # Increase this to have a larger vector. This will mean more differentiation. 
alpha = 0.025


The next step is to train the model. As before we will be using a `model.train` function to run the training process. Take a look at the [documentation](https://radimrehurek.com/gensim/models/doc2vec.html) for more information about how to train the Doc2vec model. 

In [41]:

model = Doc2Vec(vector_size =vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data, # tagged data to be used here
                total_examples=model.corpus_count,
                epochs=100)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")


C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration

### Evaluating doc2vec model

In [42]:
from gensim.models.doc2vec import Doc2Vec
model= Doc2Vec.load("d2v.model")

In [43]:
test_data = word_tokenize("How much is the price?".lower())

We can use the `model.infer_vector` function to infer the vector associated with a document. We can then use a function called `most_similar` to find the most similar vectors to the one we have created. What are the results? 

In [44]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 0.2015224  -0.23150712 -0.3772632   0.9731093   0.02674857  0.30640373
  0.35290086 -0.17238572 -0.47018495  0.08898254 -0.3221872  -0.11376996
  0.13277172 -0.34268722 -0.38118112  0.45459187 -1.005438    0.04126304
  0.54858303 -0.6681078 ]


In [45]:
similar_doc = model.docvecs.most_similar(positive = [v1], topn = 4) #positive is an attribute that shows positive correlation first followed by the correlation value
print(similar_doc)

[('6', 0.6243981122970581), ('8', 0.5830704569816589), ('22', 0.5722140073776245), ('24', 0.5691563487052917)]


In [46]:
num,_ = similar_doc[0]
num = int(num)
tagged_data[num]


TaggedDocument(words=['how', 'many', 'suites', 'are', 'there', 'in', 'hotel', 'xcaret', 'mexico', ',', 'hotel', 'xcaret', 'arte', 'and', 'la', 'casa', 'de', 'la', 'playa', '?'], tags=['6'])

Here it is very clear from the output of the previous code block that this model is not as effective as one might have thought. Are there any guesses why this model failed where we expected it to succeed?

Click on this [link](https://stackoverflow.com/questions/58206571/doc2vec-find-the-similar-sentence) to learn more about this. The gist of the issue is this

> Doc2Vec isn't going to give good results on toy-sized datasets, so you shouldn't expect anything meaningful until using much more data.

### Using pretrained doc2vec

And so let us use a pretrained model downloaded from the internet to optimize our use case. The model we have acquired is trained on the associated press news and can be downloaded [here](https://github.com/jhlau/doc2vec).

Let us load the model and evaluate it again.

In [47]:
model= Doc2Vec.load(os.getcwd()+r"\\doc2vec.bin")

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:742: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [48]:
test_data = word_tokenize("How much is the price?".lower())

In [49]:
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

V1_infer [ 0.04594279 -0.09332872  0.12052081 -0.00430305  0.05193945  0.12344071
 -0.20036738 -0.16266865  0.10080659  0.06089645  0.17401664  0.09388433
 -0.1175189  -0.13436416 -0.18134142  0.12421092 -0.19823492  0.08593023
  0.0094606  -0.04507516 -0.23374364 -0.04155522  0.06152843  0.05250691
  0.08372033 -0.21819888 -0.06024333 -0.15949504 -0.10761306  0.08231125
 -0.1787988  -0.02322111 -0.01185284  0.04147323  0.08291915  0.05479727
  0.00925557 -0.06143322  0.03315309 -0.09962466  0.02639398  0.0274008
  0.20581658  0.16853265 -0.14157169 -0.18239558  0.0885706   0.05721233
 -0.07254339  0.0779999  -0.14986704 -0.01384519  0.02613728  0.01657231
 -0.06448152 -0.13853844  0.20865412  0.04948533  0.08210691 -0.15983054
  0.07421409  0.00421122 -0.12554426  0.1676065   0.19656971  0.0842042
 -0.15895638 -0.24953152  0.2597829  -0.00240057 -0.120561    0.14736547
 -0.07604281 -0.11051979 -0.14823443  0.02896312  0.03998013  0.20227018
 -0.23322019  0.01056932  0.12341718 -0.2163

In [50]:
for i in sent_tokens:
    v2 = model.infer_vector(word_tokenize(i.lower()))
    print(i)
    print(cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1)))

which payment methods are accepted at hotel xcaret mexico, hotel xcaret arte and la casa de la playa?
[[0.43621233]]
is my credit card data safe at the time of booking my reservation?
[[0.45524934]]
could i pay for my reservation at the time of check-in at the hotel?
[[0.50117695]]
will my credit card be charged at the time of booking my reservation on the website or on the phone?
[[0.4393093]]
where is hotel xcaret mexico, hotel xcaret arte and la casa de la playa located?
[[0.505417]]
are hotel xcaret mexico, hotel xcaret arte and la casa de la playa beach front?
[[0.4301231]]
how many suites are there in hotel xcaret mexico, hotel xcaret arte and la casa de la playa?
[[0.52715945]]
are hotel xcaret mexico, hotel xcaret arte and la casa de la playa disabled friendly?
[[0.37923807]]
do hotel xcaret mexico, hotel xcaret arte and la casa de la playa have an adults only section?
[[0.42074046]]
what time is the check-in and check-out at hotel xcaret mexico, hotel xcaret arte and la casa d

In [51]:
def calc_prob(v1, q):
    probs = dict()
    for i in q:
        v2 = model.infer_vector(word_tokenize(i.lower()))
        sim = cosine_similarity(v1.reshape(1, -1),v2.reshape(1, -1))
        #print(i)
        #print(sim)
        probs[i] = sim[0][0]
    sorted_d = dict( sorted(probs.items(), key=operator.itemgetter(1),reverse=True))
    
    return list(sorted_d.items())[0]

In [52]:
calc_prob(v1, sent_tokens)

('what is the maximum capacity of each suite?', 0.67376584)

Since it appears that this model is more effective lets incorporate it into our chatbot.


In [53]:
flag=True
print("Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Jane: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Jane: "+greeting(user_response))

            else:
                print("Jane: ",end="")
                resp= calc_prob(model.infer_vector(word_tokenize(user_response)), sent_tokens)
                print(res[resp[0]], )
                print(' (With similarity of ',resp[1],')')

    else:
        flag=False
        print("Jane: Bye! take care..")

Jane: My name is Jane. I will answer your queries about this hotel. If you want to exit, type Bye!
where is xcaret?
Jane: Hotel Xcaret Arte is located in front of the Caribbean Sea, next to Hotel Xcaret Mexico 15 minutes away from Playa del Carmen 45 minutes away from Cancun International Airport 45 minutes away.
 (With similarity of  0.72526443 )
what is the maximum capacity?
Jane: The suites have a maximum capacity of 3 people, depending on the category, In the Master Suite category 5 people are allowed, and in the Presidential Suite, up to 6 people.
 (With similarity of  0.87497884 )
Which types of payments you accept?
Jane: At the time of booking we will provide you a section where you will be able to send us your requests, Please note that any request will be subject to availability and/or may result in additional charges.
 (With similarity of  0.59951556 )
What are the payment methods?
Jane: We accept Visa, MasterCard and American Express credit cards, if you need to pay with two

**After observing the performance** of both of the models there are some clear conclusions that we can draw from this:

1. The Doc2vec model requires a lot more data for it to understand the relationship between words. And even after using a pre-trained model there is a clear lack of quality in the responses from the model compared to the cosine similarity model. 
2. The cosine similarity model works better for a smaller and more well defined dataset. This means that a few simple questions can be solved effectively but complex questions that require context wont be solved by it. 

Questions and answers reference >> https://www.hotelxcaret.com/en/faqs/